In [64]:
import re
import numpy as np
import pandas as pd
import feather
import xgboost as xgb
import feather
from sklearn.base import BaseEstimator as be
from sklearn.base import TransformerMixin as tm
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

from sklearn.cross_validation import *
from sklearn.grid_search import GridSearchCV

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

from sklearn.utils import shuffle

import pickle

In [2]:

def save_classifier(fname, clf):
    # save the classifier
    with open(fname, 'wb') as fid:
        pickle.dump(clf, fid)

def load_classifier(fname):
    # load it again
    with open(fname, 'rb') as fid:
        clf = pickle.load(fid)
        return clf

In [3]:
# Any results you write to the current directory are saved as output.
submission = pd.read_csv('../data/stage2_sample_submission.csv')
stage1_test = pd.read_csv('../data/test_variants')
stage2_test = pd.read_csv('../data/stage2_test_variants.csv')
stage1_solution = pd.read_csv('../data/stage1_solution_filtered.csv')

stage1_solution = stage1_solution.merge(stage1_test, how = 'left', on = 'ID')



In [4]:
df_train = feather.read_dataframe('../cache/train_stage2_fe.feather')

In [5]:
train_ids = df_train.ID.values

In [6]:
df_train = df_train.drop('ID', axis=1)

In [7]:
y = df_train['Class'].values

In [8]:
df_test = feather.read_dataframe('../cache/test_stage2_fe.feather')
pid = df_test.ID
df_test = df_test.drop('ID', axis=1)
# df_test['Class'] = y_hack

In [9]:
df_train.shape

(3689, 3590)

In [10]:
df_train.head()

,Gene,Text,Variation,GeneVar,Gene_Share,Variation_Share,Gene_Variation_Share,GL,VL,Gene_0,...,Gene_words,Text_len,Text_words,Variation_lbl_enc,Variation_len,Variation_words,GeneVar_lbl_enc,GeneVar_len,GeneVar_words,Class
0,FAM58A,Cyclin-dependent kinases (CDKs) regulate a var...,Truncating Mutations,FAM58A Truncating Mutations,1,1,2,6,20,5,...,1,39672,6105,3460,20,2,1654,27,3,1
1,CBL,Abstract Background Non-small cell lung canc...,W802*,CBL W802*,1,1,2,3,5,2,...,1,36691,5783,3748,5,1,941,9,2,2
2,CBL,Abstract Background Non-small cell lung canc...,Q249E,CBL Q249E,1,1,2,3,5,2,...,1,36691,5783,2425,5,1,933,9,2,2
3,CBL,Recent evidence has demonstrated that acquired...,N454D,CBL N454D,1,1,2,3,5,2,...,1,36238,5625,2132,5,1,929,9,2,3
4,CBL,Oncogenic mutations in the monomeric Casitas B...,L399V,CBL L399V,1,1,2,3,5,2,...,1,41308,6248,1854,5,1,927,9,2,4


In [11]:
df_test = feather.read_dataframe('../cache/test_stage2_fe.feather')

In [12]:
df_test.shape

(986, 3590)

In [13]:
df_test.head()

,Gene,ID,Text,Variation,GeneVar,Gene_Share,Variation_Share,Gene_Variation_Share,GL,VL,...,Gene_len,Gene_words,Text_len,Text_words,Variation_lbl_enc,Variation_len,Variation_words,GeneVar_lbl_enc,GeneVar_len,GeneVar_words
0,CHEK2,1,The incidence of breast cancer is increasing i...,H371Y,CHEK2 H371Y,1,1,2,5,5,...,5,1,33403,4991,1379,5,1,1080,11,2
1,AXIN2,2,An unselected series of 310 colorectal carcino...,Truncating Mutations,AXIN2 Truncating Mutations,1,1,2,5,20,...,5,1,66400,10348,3460,20,2,286,26,3
2,WNT4,3,Mycosis fungoides and Sézary syndrome are prim...,E216G,WNT4 E216G,0,1,1,4,5,...,4,1,58544,8638,672,5,1,4287,10,2
3,SUCLA2,4,Regulated progression through the cell cycle ...,G118R,SUCLA2 G118R,0,1,1,6,5,...,6,1,42023,6221,1038,5,1,3841,12,2
4,BRAF,5,Pilocytic astrocytoma (PA) is emerging as a tu...,T599insTT,BRAF T599insTT,1,0,1,4,9,...,4,1,22499,3280,3402,9,1,430,14,2


In [14]:
########################################
## process texts in datasets
########################################


# The function "text_to_wordlist" is from
# https://www.kaggle.com/currie32/quora-question-pairs/the-importance-of-cleaning-text
def text_to_wordlist(text, remove_stopwords=True, stem_words=True):
    # Clean the text, with the option to remove stopwords and to stem words.
    
    # Convert words to lower case and split them
    text = text.lower().split()

    my_stopwords = [
        "fig", "figure", "et", "al", "table",
        "data", "analysis", "analyze", "study",
        "method", "result", "conclusion", "author",
        "find", "found", "show", "perform",
        "demonstrate", "evaluate", "discuss"
    ]
    
    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = [w for w in text if not w in my_stopwords]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)

In [15]:
# https://www.kaggle.com/the1owl/redefining-treatment-0-57456
class cust_regression_vals(be, tm):
    def fit(self, x, y=None):
        return self
    def transform(self, x):
        x = x.drop(['Gene', 'Variation', 'ID','Text', 'GeneVar'],axis=1).values
        return x

class cust_txt_col(be, tm):
    def __init__(self, key):
        self.key = key
    def fit(self, x, y=None):
        return self
    def transform(self, x):
        return x[self.key].apply(str)


In [16]:
# from https://www.kaggle.com/the1owl/redefining-treatment-0-57456
fp = Pipeline([
    ('union', FeatureUnion(
        n_jobs = -1,
        transformer_list = [
            ('standard', cust_regression_vals()),
            ('pi1', Pipeline([('Gene', cust_txt_col('Gene')), 
                                       ('count_Gene', CountVectorizer(analyzer=u'char',ngram_range=(1, 3))), 
                                       ('tsvd1', TruncatedSVD(n_components=20, n_iter=25, random_state=12))])),
            ('pi2', Pipeline([('Variation', cust_txt_col('Variation')), 
                                       ('count_Variation', CountVectorizer(analyzer=u'char',ngram_range=(1, 3))), 
                                       ('tsvd2', TruncatedSVD(n_components=20, n_iter=25, random_state=12))])),
            ('pi3', Pipeline([('GeneVar', cust_txt_col('GeneVar')), 
                                       ('count_GeneVar', CountVectorizer(analyzer=u'char', ngram_range=(1, 3))), 
                                       ('tsvd2', TruncatedSVD(n_components=20, n_iter=25, 
                                                                            random_state=12))])),
            ('pi4',Pipeline([('Text', cust_txt_col('Text')), 
                                       ('hv', HashingVectorizer(decode_error='ignore', 
                                                                                        n_features=2 ** 16,
                                                                                        non_negative=True, 
                                                                                        ngram_range=(1, 5))),
                                       ('tfidf_Text', TfidfTransformer()), 
                                       ('tsvd3', TruncatedSVD(n_components=300, n_iter=25, 
                                                                            random_state=12))]))

        
        ])
    )])


In [17]:
df_train['Text'] = [text_to_wordlist(w) for w in df_train['Text'].values]

In [18]:
df_test['Text'] = [text_to_wordlist(w) for w in df_test['Text'].values]

In [19]:
df_train.head()

,Gene,Text,Variation,GeneVar,Gene_Share,Variation_Share,Gene_Variation_Share,GL,VL,Gene_0,...,Gene_words,Text_len,Text_words,Variation_lbl_enc,Variation_len,Variation_words,GeneVar_lbl_enc,GeneVar_len,GeneVar_words,Class
0,FAM58A,cyclin - depend kinas cdks regul varieti funda...,Truncating Mutations,FAM58A Truncating Mutations,1,1,2,6,20,5,...,1,39672,6105,3460,20,2,1654,27,3,1
1,CBL,abstract background non - small cell lung canc...,W802*,CBL W802*,1,1,2,3,5,2,...,1,36691,5783,3748,5,1,941,9,2,2
2,CBL,abstract background non - small cell lung canc...,Q249E,CBL Q249E,1,1,2,3,5,2,...,1,36691,5783,2425,5,1,933,9,2,2
3,CBL,recent evid demonstr acquir uniparent disomi a...,N454D,CBL N454D,1,1,2,3,5,2,...,1,36238,5625,2132,5,1,929,9,2,3
4,CBL,oncogen mutat monomer casita b - lineag lympho...,L399V,CBL L399V,1,1,2,3,5,2,...,1,41308,6248,1854,5,1,927,9,2,4


In [20]:
y = df_train['Class'].values

In [21]:
df_train['ID'] = [i for i in range(df_train.shape[0])]

In [22]:
tr = df_train
te = df_test

In [23]:
# y = tr['Class'].values

In [24]:
set(te.columns) - set(tr.columns)

set()

In [25]:
df_train = df_train.drop('Class',axis=1)

In [26]:
# df_train = df_train.drop('ID', axis=1)
# df_test = df_test.drop('ID', axis=1)

In [27]:
# df_train = tr

In [28]:
df_train.head()

,Gene,Text,Variation,GeneVar,Gene_Share,Variation_Share,Gene_Variation_Share,GL,VL,Gene_0,...,Gene_words,Text_len,Text_words,Variation_lbl_enc,Variation_len,Variation_words,GeneVar_lbl_enc,GeneVar_len,GeneVar_words,ID
0,FAM58A,cyclin - depend kinas cdks regul varieti funda...,Truncating Mutations,FAM58A Truncating Mutations,1,1,2,6,20,5,...,1,39672,6105,3460,20,2,1654,27,3,0
1,CBL,abstract background non - small cell lung canc...,W802*,CBL W802*,1,1,2,3,5,2,...,1,36691,5783,3748,5,1,941,9,2,1
2,CBL,abstract background non - small cell lung canc...,Q249E,CBL Q249E,1,1,2,3,5,2,...,1,36691,5783,2425,5,1,933,9,2,2
3,CBL,recent evid demonstr acquir uniparent disomi a...,N454D,CBL N454D,1,1,2,3,5,2,...,1,36238,5625,2132,5,1,929,9,2,3
4,CBL,oncogen mutat monomer casita b - lineag lympho...,L399V,CBL L399V,1,1,2,3,5,2,...,1,41308,6248,1854,5,1,927,9,2,4


In [29]:
df_train.shape

(3689, 3590)

In [30]:
df_train = fp.fit_transform(df_train)
print (df_train.shape)


/home/watts/anaconda2/envs/aind-dog/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)
/home/watts/anaconda2/envs/aind-dog/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


(3689, 3945)


In [31]:
df_test = fp.fit_transform(df_test)
print (df_test.shape)

/home/watts/anaconda2/envs/aind-dog/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)
/home/watts/anaconda2/envs/aind-dog/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


(986, 3945)


In [55]:
np.save('../cache/train_stage2_fe2', df_train)

In [56]:
np.save('../cache/test_stage2_fe2', df_test)

In [32]:
y = y - 1 #fix for zero bound array

In [33]:
print(y.shape)

(3689,)


In [34]:
np.unique(y)

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [35]:
# parameters = {
#   "n_estimators": [5, 50, 200],
#   "max_depth" : [None, 10,20],
#   "min_samples_split" : [2, 5, 10],
# }

param_grid = { 
    'n_estimators': [50, 200, 700],
    'max_features': ['auto', 'sqrt', 'log2']
}

for cls in [0,1,2,3,4,5,6,7,8]:
    y_this_class = [1 if cls==i else 0 for i in y]
#     print(y_this_class)
    model_RF = RandomForestClassifier(random_state=0)
    clf = GridSearchCV(model_RF, param_grid=param_grid, cv=5,n_jobs=4)
    clf.fit(df_train, y_this_class )
    fname = '../cache/rf_cls_' + str(cls) + '.pkl'
    save_classifier(fname, clf)

In [45]:
n_classes = np.unique(y)

In [51]:
df = pd.DataFrame()
for cls in n_classes:
    fname = '../cache/rf_cls_' + str(cls) + '.pkl'
    clf = load_classifier(fname)
    print(clf.best_score_)
    print(clf.best_params_)
    test_probs = clf.predict_proba(df_test)[:,1]
    df['class' + str(cls+1)] = test_probs

0.8484684196259149
{'max_features': 'auto', 'n_estimators': 200}
0.855245323936026
{'max_features': 'log2', 'n_estimators': 50}
0.9666576307942532
{'max_features': 'log2', 'n_estimators': 50}
0.8102466793168881
{'max_features': 'log2', 'n_estimators': 200}
0.8910273786934129
{'max_features': 'log2', 'n_estimators': 50}
0.9330441854161019
{'max_features': 'log2', 'n_estimators': 50}
0.75060992138791
{'max_features': 'auto', 'n_estimators': 50}
0.9943074003795066
{'max_features': 'auto', 'n_estimators': 50}
0.9894280292762266
{'max_features': 'auto', 'n_estimators': 50}


In [52]:
df['ID'] = pid

In [53]:
df.head()

,class1,class2,class3,class4,class5,class6,class7,class8,class9,ID
0,0.210,0.20,0.04,0.250,0.06,0.08,0.20,0.02,0.00,1
1,0.570,0.30,0.02,0.170,0.02,0.06,0.24,0.04,0.04,2
2,0.225,0.20,0.04,0.250,0.00,0.10,0.32,0.00,0.02,3
3,0.220,0.22,0.00,0.245,0.14,0.12,0.24,0.06,0.00,4
4,0.175,0.12,0.02,0.160,0.08,0.04,0.28,0.02,0.00,5


In [54]:
df.to_csv('../submissions/submission_staeg2_rf.csv', index=False)


In [ ]:
# scored 2.18400 on stage2 public leaderboard, and 8.10579 on private leaderboard


In [57]:
df1 = pd.DataFrame()
df1['ID'] = pid
df1.to_csv('../cache/test_id.csv', index=False)

In [62]:
df = pd.DataFrame()
df['y'] = y
df.to_csv('../cache/train_stage2_labels.csv', index=False)

In [66]:
X = df_train
Y = y
Z = df_test


In [68]:
Y1 = Y.T

In [75]:
y2 = shuffle(Y, random_state=1)
y3 = shuffle(Y, random_state=2)
y4 = shuffle(Y, random_state=3)
y5 = shuffle(Y, random_state=4)
y6 = shuffle(Y, random_state=5)
y7 = shuffle(Y, random_state=6)
y8 = shuffle(Y, random_state=7)
y9 = shuffle(Y, random_state=8)

Y1 = np.vstack((Y, y2, y3, y4, y5, y6, y7, y8, y9)).T

In [76]:
forest = RandomForestClassifier(n_estimators=1000, random_state=1)
multi_target_forest = MultiOutputClassifier(forest, n_jobs=-1)

In [77]:

preds = multi_target_forest.fit(X, Y1).predict(Z)

In [78]:
preds.shape

(986, 9)

In [79]:
print(preds)

[[3 6 6 ..., 6 6 6]
 [0 6 6 ..., 6 6 6]
 [6 6 6 ..., 6 6 6]
 ..., 
 [6 6 6 ..., 6 6 6]
 [3 6 6 ..., 6 6 6]
 [6 6 6 ..., 6 6 6]]


In [98]:
preds_proba = multi_target_forest.fit(X, Y1).predict_proba(Z)

In [100]:
preds_proba[0].shape

(986, 9)

In [108]:
preds_proba[0]

array([[ 0.206,  0.144,  0.033, ...,  0.192,  0.014,  0.019],
       [ 0.547,  0.113,  0.006, ...,  0.14 ,  0.006,  0.006],
       [ 0.161,  0.191,  0.022, ...,  0.241,  0.013,  0.017],
       ..., 
       [ 0.14 ,  0.213,  0.023, ...,  0.3  ,  0.007,  0.013],
       [ 0.177,  0.174,  0.023, ...,  0.223,  0.012,  0.014],
       [ 0.191,  0.174,  0.021, ...,  0.231,  0.014,  0.01 ]])

In [ ]:
preds_proba[0]

In [116]:
df = pd.DataFrame(preds_proba[0], 
                  columns=['class1', 'class2', 'class3', 'class4', 'class5', 'class6', 'class7', 'class8', 'class9'])


In [117]:
df['ID'] = pid
df.to_csv('../submissions/sub_stage2_multi_rf.csv', index=False)

In [ ]:
# scored 2.64206 on stage2 private LB, 1.54953 on stage2 public LB

In [114]:
save_classifier('../cache/stage2_multi_rf.pkl', multi_target_forest)

In [118]:
df.head()

,class1,class2,class3,class4,class5,class6,class7,class8,class9,ID
0,0.206,0.144,0.033,0.241,0.069,0.082,0.192,0.014,0.019,1
1,0.547,0.113,0.006,0.114,0.015,0.053,0.140,0.006,0.006,2
2,0.161,0.191,0.022,0.233,0.054,0.068,0.241,0.013,0.017,3
3,0.202,0.129,0.021,0.272,0.053,0.059,0.229,0.010,0.025,4
4,0.173,0.245,0.016,0.130,0.051,0.067,0.301,0.009,0.008,5


In [119]:
df.shape

(986, 10)